## **Forest Structure Analysis & Classification Using ML**

### **About the Dataset**

**Dataset**: [Tree Inventory and Biometry Measurements, Tapajos National Forest, Para, Brazil, 2010](https://daac.ornl.gov/VEGETATION/guides/Forest_Inventory_Tapajos.html)

**Overview** </br>

This dataset includes information on tree inventory, height, diameter at breast height (DBH), and crown measurements from 30 plots in the Tapajos National Forest in Para, Brazil, collected in September 2010. The plots are divided among primary forest, primary forest that underwent selective logging from 1999 to 2003, and secondary forest with varying ages and disturbance histories. </br></br>

Out of the 50 x 50-meter plots, eight were in primary forests, eight were in logged primary forests, and fourteen were in secondary forests. Trees with a diameter of 5 cm or more in early successional stands and 10 cm or more in other stands were measured and identified by species or genus. DBH was recorded to the nearest 0.1 cm, while heights and crown depths were visually estimated and calibrated using a laser rangefinder. Crown radius was measured in two directions using a tape measure. </br></br>

The Tapajos National Forest is situated about 50 km south of Santarem, Pará, along highway BR-163. </br></br>

This study uses the comma-separated (.csv) file with with the inventory and biometric measurements of trees.

**Columns**: The table below lists the columns in `Tapajos_inventory_data_2010.csv` along with their corresponding units.


| **Column Name**                  | **Units**     | **Description**                                                                 |
|----------------------------------|---------------|---------------------------------------------------------------------------------|
| `plot`                           | —             | Unique plot identifier                                                         |
| `type`                           | —             | Forest type: `SF` = Secondary, `SLF` = Selectively Logged, `PF` = Primary      |
| `origin`                         | —             | Plot corner used as the origin for local coordinates (`NE` or `SW`)            |
| `tree`                           | —             | Tree identification number                                                     |
| `family`                         | —             | Scientific family classification of the tree                                   |
| `genus`                          | —             | Genus classification                                                           |
| `scientific_name`               | —             | Full scientific name (Genus + species)                                         |
| `common_name`                    | —             | Local/common name of the tree                                                  |
| `density_wood`                  | g/cm³         | Wood density (from literature)                                                 |
| `dbh`                            | cm            | Diameter at breast height (1.3 m above ground)                                 |
| `ht_crown_base`                 | m             | Height to the base of the crown                                                |
| `ht_total`                       | m             | Total tree height                                                              |
| `depth_crown`                   | m             | Depth of the crown (ht_total - ht_crown_base)                                  |
| `stem_x`                         | m             | X-coordinate of the tree stem position                                         |
| `stem_y`                         | m             | Y-coordinate of the tree stem position                                         |
| `crown_radius_x0`               | m             | Crown radius from origin to stem along the X-axis                              |
| `crown_radius_x50`              | m             | Crown radius from stem to opposite edge along the X-axis                       |
| `crown_radius_y0`               | m             | Crown radius from origin to stem along the Y-axis                              |
| `crown_radius_y50`              | m             | Crown radius from stem to opposite edge along the Y-axis                       |
| `ht_crown_max_x0`              | m             | Height of maximum crown width at edge closest to origin (X-axis)               |
| `ht_crown_max_x50`             | m             | Height of maximum crown width at far edge from origin (X-axis)                 |
| `ht_crown_max_y0`              | m             | Height of maximum crown width at edge closest to origin (Y-axis)               |
| `ht_crown_max_y50`             | m             | Height of maximum crown width at far edge from origin (Y-axis)                 |
| `shape_coeff_crown_x0`         | unitless       | Shape coefficient above max crown radius, edge near origin (X-axis)           |
| `shape_coeff_crown_x50`        | unitless       | Shape coefficient above max crown radius, edge far from origin (X-axis)       |
| `shape_coeff_crown_y0`         | unitless       | Shape coefficient above max crown radius, edge near origin (Y-axis)           |
| `shape_coeff_crown_y50`        | unitless       | Shape coefficient above max crown radius, edge far from origin (Y-axis)       |
| `shape_coeff_crown_below_x0`   | unitless       | Shape coefficient below max crown radius, edge near origin (X-axis)           |
| `shape_coeff_crown_below_x50`  | unitless       | Shape coefficient below max crown radius, edge far from origin (X-axis)       |
| `shape_coeff_crown_below_y0`   | unitless       | Shape coefficient below max crown radius, edge near origin (Y-axis)           |
| `shape_coeff_crown_below_y50`  | unitless       | Shape coefficient below max crown radius, edge far from origin (Y-axis)       |


### **Research Question**

> What are the main differences in forest structure and tree types between primary, logged, and secondary
forests?



This research question aims to identify and compare differences in forest structure (such as tree height, diameter, crown dimensions) and species composition (e.g., tree family, genus) among primary (undisturbed), selectively logged, and secondary (regenerating) forests. Furthermore, three different prediction models will be trained comparing their performance on the given data.

### **Imports**

Import libraries needed.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from sklearn.inspection import permutation_importance

### **Load Dataset**
Load the CSV file into a DataFrame named df.

Make sure to download the dataset and upload it under the Files (the one with the Folder Icon) of Google Colab.




In [ ]:
df = pd.read_csv('Tapajos_inventory_data_2010.csv')

Display the first five rows of the DataFrame using **`.head(5)`** to get an initial overview of the data.


In [ ]:
df.head(5)

,plot,type,origin,tree,family,genus,scientific_name,common_name,density_wood,dbh,...,ht_crown_max_y0,ht_crown_max_y50,shape_coeff_crown_x0,shape_coeff_crown_x50,shape_coeff_crown_y0,shape_coeff_crown_y50,shape_coeff_crown_below_x0,shape_coeff_crown_below_x50,shape_coeff_crown_below_y0,shape_coeff_crown_below_y50
0,1,SF,NE,1,Hypericaceae,Vismia,Vismia guianensis,Lacre branco,0.480,5.9,...,4.5,6.0,4.0,2.0,0.4,4.0,0.8,0.4,0.3,0.8
1,1,SF,NE,2,Salicaceae,Banara,Banara nitida,Cabelo de cutia,0.600,7.4,...,-9999.0,7.1,1.3,1.6,-9999.0,1.6,0.8,0.6,-9999.0,0.6
2,1,SF,NE,3,Salicaceae,Banara,Banara nitida,Cabelo de cutia,0.600,6.3,...,-9999.0,7.1,8.0,-9999.0,-9999.0,8.0,8.8,-9999.0,-9999.0,8.8
3,1,SF,NE,4,Salicaceae,Banara,Banara nitida,Cabelo de cutia,0.600,5.4,...,-9999.0,2.3,1.0,-9999.0,-9999.0,1.3,0.3,-9999.0,-9999.0,0.3
4,1,SF,NE,5,Chrysobalanaceae,Couepia,Couepia sp.,Macucu,0.791,8.1,...,5.8,9.3,2.7,2.0,2.7,8.0,6.0,0.8,0.8,0.8


Use **`.info()`** to display a summary of the DataFrame, including data types, non-null counts, and memory usage.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3390 entries, 0 to 3389
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   plot                         3390 non-null   int64  
 1   type                         3390 non-null   object 
 2   origin                       3300 non-null   object 
 3   tree                         3390 non-null   int64  
 4   family                       3390 non-null   object 
 5   genus                        3390 non-null   object 
 6   scientific_name              3390 non-null   object 
 7   common_name                  3390 non-null   object 
 8   density_wood                 3390 non-null   float64
 9   dbh                          3390 non-null   float64
 10  ht_crown_base                3390 non-null   float64
 11  ht_total                     3390 non-null   float64
 12  depth_crown                  3390 non-null   float64
 13  stem_x            

The output of **.info()** shows that the dataset contains 31 columns, indexed from 0 to 30. Most columns have non-null values and are of the **`float64`** data type.Except **`type`, `origin`, `family`, `genus`, `scientific_name`**, and **`common_name`**, which are of **`object`** (string) type, as well as **`plot`** and **`tree`**, which are of **`int64`** type.

Use **`.describe()`** to generate basic statistical summaries for numeric columns, such as mean, standard deviation, and quartiles.

In [ ]:
df.describe()

Lastly, use **`.shape`** to display the dimensions of the DataFrame (rows, columns), to understand its size.

In [ ]:
df.shape

### **Data Preprocessing**
In this section, the data will be prepared for analysis by performing data preprocessing steps.

#### Missing Values

In [ ]:
# Check number of null values in the DataFrame.
print(df.isnull().sum())

The output shows that there are 90 rows that has missing values for the column **`origin`**.

In [ ]:
# Check count of each unique value in origin column
df['origin'].value_counts()

The value counts for each unique value in the origin column does not total to 3390. Next we will check for columns with value **`-9999`** which indicates missing or data is not provided.

In [ ]:
# Check which column or row has the value -9999 indicating missing or data that is not provided.
(df == -9999).sum()

As seen from the table above, the columns related to tree crowns have more than 50% missing values. For now, only rows with missing origin will be dropped.

In [ ]:
# drop rows with missing origin
df = df.dropna()

In [ ]:
# Check number of null values in the DataFrame.
print(df.shape)
print(df.isnull().sum())

### Duplicates

It was observed that the same tree may have multiple records in the dataset. To confirm this, duplicates were checked based on tree ID numbers using **`tree`** column.

In [ ]:
# Check for duplicates in tree column
df['tree'].duplicated().sum()

A **`tree`** ID seems to be duplicated within the column. Now we check **`tree`** ID per **`plot`.**

In [ ]:
# Check for duplicates with plot and
df[['plot', 'tree']].duplicated().sum()

This shows that the tree id is not globally unique, but it is unique per plot.

In [ ]:
# Check number of unique tree id
df['tree'].nunique()

The cell below drops duplicates in the dataframe.

In [ ]:
# Drop Duplicates
df = df.drop_duplicates()
print(df.shape)

### **Exploratory Data Analysis**

**1. Distribution of Numerical Columns**

Histograms are used to visualize the distribution of columns containing numerical values, allowing us to observe patterns, skewness, and potential outliers.

In [ ]:
numeric_cols = df.select_dtypes(include='number').columns

# Plot histograms
df[numeric_cols].hist(figsize=(15, 12), bins=30, edgecolor='black')
plt.tight_layout()
plt.suptitle('Histograms of Numerical Columns', fontsize=16, y=1.02)
plt.show()

The histograms reveal that most numerical features like `dbh` and `ht_total` are right-skewed, indicating a prevalence of smaller trees with fewer larger ones. `density_wood` shows a more symmetrical distribution, while crown-related features displayed a significant number of -9999 values, confirming extensive missing data. This analysis highlights the typical structure of the forest data and identifies columns with data quality issues.

**2. Distribution of Categorical Columns**

Bar plots are used to visualize the distribution of categorical columns. This helps identify the frequency of each category and detect any imbalances or dominant classes in the data.

In [ ]:
categorical_cols = df.select_dtypes(include='object').columns

# Loop through and plot countplots for each, showing top 10 categories
for col in categorical_cols:
    plt.figure(figsize=(10, 5))

    top_categories = df[col].value_counts().nlargest(10).index
    sns.countplot(data=df[df[col].isin(top_categories)], x=col, order=top_categories)

    plt.xticks(rotation=45)
    plt.title(f'Distribution of Top 15 {col}')
    plt.tight_layout()
    plt.show()

The bar plots show that Secondary Forest (SF) is the most frequent forest type, indicating an imbalance in the dataset's distribution across forest types. The `origin` column reveals that "SW" is a more common origin point than "NE". Additionally, the plots for `family`, `genus`, and `scientific_name` highlight the most abundant tree classifications, providing insight into the dominant tree types in the sampled areas.

**3. Tree Height and Diameter per Forest Type**

In [ ]:
df.groupby('type')[['ht_total', 'dbh']].describe()

In [ ]:
# Tree Height by Forest Type (Boxplot)

plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='type', y='ht_total')
plt.title('Tree Height by Forest Type')
plt.xlabel('Forest Type')
plt.ylabel('Total Tree Height (m)')
plt.tight_layout()
plt.show()

This boxplot visually compares the distribution of tree heights across the three forest types. It shows that Primary (PF) and Selectively Logged (SLF) forests generally have taller trees with higher median heights and a greater range of heights compared to Secondary Forests (SF). The SF type displays a lower median height and a more condensed distribution, indicating that trees in secondary forests are typically shorter.

In [ ]:
# DBH by Forest Type (Boxplot)
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='type', y='dbh')
plt.title('Tree Diameter (DBH) by Forest Type')
plt.xlabel('Forest Type')
plt.ylabel('DBH (cm)')
plt.tight_layout()
plt.show()

This boxplot illustrates the distribution of tree diameters at breast height (DBH) for each forest type. Similar to tree height, trees in Primary (PF) and Selectively Logged (SLF) forests exhibit larger DBH values, with higher medians and more variability than trees in Secondary Forests (SF). The SF type's boxplot is lower and narrower, reflecting that trees in secondary forests generally have smaller diameters.

**4. Tree Density per Plot**

In [ ]:
tree_counts = df.groupby(['plot', 'type']).size().reset_index(name='tree_count')
tree_counts.groupby('type')['tree_count'].describe()

The descriptive statistics for tree density per plot show that Secondary Forests (SF) have the highest average tree count per plot, with a mean of 134.36, and also the widest range and standard deviation in tree density. Selectively Logged Forests (SLF) have a slightly higher average tree count (99.29) compared to Primary Forests (PF) (90.50). Both PF and SLF show less variability in tree density across their plots compared to SF.

**5. Compare Crown Depth and Shape**

In [ ]:
df.groupby('type')[['depth_crown', 'shape_coeff_crown_x0']].mean()

The table shows the average crown depth and the average of one of the shape coefficients (`shape_coeff_crown_x0`) for each forest type. While there are differences in the mean crown depth across the forest types (PF and SLF have larger average depths than SF), the mean values for the shape coefficient are heavily skewed by the large number of -9999 values. Therefore, based on this specific output, we can only meaningfully compare the average crown depth, which is higher in primary and selectively logged forests.

**6. Species Richness (Species per Forest Type)**

In [ ]:
species_richness = df.groupby('type')['scientific_name'].nunique().reset_index()
print(species_richness)

In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(data=species_richness, x='type', y='scientific_name')
plt.title('Species Richness by Forest Type')
plt.ylabel('Number of Unique Species')
plt.xlabel('Forest Type')
plt.tight_layout()
plt.show()

The analysis of species richness shows the number of unique tree species identified in each forest type. Selectively Logged Forests (SLF) have the highest number of unique species (124), followed closely by Primary Forests (PF) with 112 unique species, and then Secondary Forests (SF) with 111. (might remove this part)This suggests that logging activities might initially increase local species diversity or that these areas retain a mix of original and colonizing species, while secondary forests are still developing their full species composition.

**7. Family Composition Comparison**

In [ ]:
df.groupby('type')['family'].value_counts(normalize=True).unstack().fillna(0)

In [ ]:
# Family Composition (Stacked Bar Plot)
top_families = df['family'].value_counts().nlargest(5).index
df_top = df[df['family'].isin(top_families)]

family_counts = pd.crosstab(df_top['type'], df_top['family'], normalize='index')

family_counts.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='tab20')
plt.title('Relative Abundance of Tree Families by Forest Type (Top 5)')
plt.ylabel('Proportion')
plt.xlabel('Forest Type')
plt.legend(title='Family', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

The analysis of family composition, particularly focusing on the top families, reveals distinct differences in the dominant tree families present in each forest type. Secondary Forests (SF) show a higher relative abundance of certain families like Salicaceae and Urticaceae compared to Primary (PF) and Selectively Logged (SLF) forests. Conversely, PF and SLF forests tend to have a higher proportion of families such as Sapotaceae and Burseraceae, indicating variations in taxonomic structure related to forest disturbance history.

**8. Wood Density by Forest Type**

In [ ]:
df.groupby('type')['density_wood'].describe()

In [ ]:
# Wood Density Distribution
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='type', y='density_wood')
plt.title('Wood Density by Forest Type')
plt.xlabel('Forest Type')
plt.ylabel('Wood Density (g/cm³)')
plt.tight_layout()
plt.show()

The analysis of wood density shows that Primary (PF) and Selectively Logged (SLF) forests have higher average wood densities compared to Secondary Forests (SF). The box plot visually confirms this, showing that PF and SLF have higher median wood densities and distributions shifted towards higher values. This suggests that trees in more mature or previously logged forests tend to have denser wood than those in regenerating secondary forests.

### **Data Preparation**

**`crown_cols`** contains measurements related to the size, shape, and structure of tree crowns in both horizontal and vertical dimensions. However, due to a high proportion of missing values, these columns will be excluded from model training.

In [ ]:
crown_cols = [
    'crown_radius_x0', 'crown_radius_x50', 'crown_radius_y0', 'crown_radius_y50',
    'ht_crown_max_x0', 'ht_crown_max_x50', 'ht_crown_max_y0', 'ht_crown_max_y50',
    'shape_coeff_crown_x0', 'shape_coeff_crown_x50', 'shape_coeff_crown_y0', 'shape_coeff_crown_y50',
    'shape_coeff_crown_below_x0', 'shape_coeff_crown_below_x50',
    'shape_coeff_crown_below_y0', 'shape_coeff_crown_below_y50'
]

df = df.drop(columns=crown_cols)

Show new df after dropping **`crown_cols`**

In [ ]:
df.head()

Next, features are selected, and the feature matrix X and target variable y are created for model training.

In [ ]:
# Select features and target
features = ['ht_total', 'dbh', 'density_wood', 'depth_crown','ht_crown_base', 'stem_x', 'stem_y','origin', 'genus', 'family']
X = df[features]
y = df['type']

## **NEW! Check for correlation of numerical features**

In [ ]:
corr = X.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='coolwarm')

In [ ]:
sns.lmplot(x="ht_total", y="ht_crown_base", data=X)  # linear regression line

In [ ]:
sns.lmplot(x="ht_total", y="depth_crown", data=X)  # linear regression line

In [ ]:
sns.lmplot(x="ht_total", y="dbh", data=X)  # linear regression line

In [ ]:
sns.lmplot(x="dbh", y="ht_crown_base", data=X)  # linear regression line

In [ ]:
sns.lmplot(x="dbh", y="depth_crown", data=X)  # linear regression line

Show feature matrix **`X`** using **`.head()`**

In [ ]:
X.head()

Encode categorical variables such as **`origin`, `genus`, and `family`** using **OneHotEncoder()**.

In [ ]:
categorical_cols = ['origin', 'genus', 'family']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded = encoder.fit_transform(X[categorical_cols])

encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(categorical_cols),
    index=df.index
)

X = pd.concat([X.drop(categorical_cols, axis=1), encoded_df], axis=1)

Show new feature matrix **`X`** after encoding categorical columns.

In [ ]:
X.head()

In [ ]:
# Handle missing values
# imputer = SimpleImputer(strategy='mean')
# X = pd.DataFrame(X, columns=features)

Next encode target variable **`y`** using **LabelEncoder()**

In [ ]:
# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # 0 = PF, 1 = SF, 2 = SLF

Split data into training and testing.

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

Lastly, the training data is scaled using **StandardScaler()** to standardize the feature values. This step is important for distance-based models like **K-Nearest Neighbors (KNN)**, which are sensitive to the scale of the input features.

In [ ]:
# Scale for KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### **Model**

In this section, the prepared data will be used to train three different models in order to compare their performance and methodologies.

### Decision Trees

In [ ]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("Decision Tree Report:\n", classification_report(y_test, y_pred_dt, target_names=le.classes_))

# Feature Importance
importance_dt = pd.Series(dt.feature_importances_, index=X.columns)

# Plot Top 10 Features
importance_dt.nlargest(10).sort_values().plot(
    kind='barh',
    title="Top 10 Feature Importances (Decision Tree)"
)

plt.tight_layout()
plt.show()

The Decision Tree achieved an overall accuracy of 77%. It performed best on classifying Secondary Forests (SF) with high precision and recall (0.93 and 0.92 respectively). However, its performance was lower for Primary Forests (PF) and Selectively Logged Forests (SLF), with f1-scores of 0.55 and 0.57, indicating more errors in identifying these forest types.

The Decision Tree model identified `dbh` and `origin_NE` as the most important features for classification. Other significant features included `ht_crown_base`, `density_wood`, and `depth_crown`, indicating that tree size and location origin are key discriminators for this model. The remaining features in the top 10 contributed less to the decision-making process.

In [ ]:
cm_dt = confusion_matrix(y_test, y_pred_dt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_dt, display_labels=le.classes_)
disp.plot(cmap="Greens")
plt.title("Decision Tree - Confusion Matrix")
plt.show()

The confusion matrix shows the Decision Tree model performed very well in classifying Secondary Forests (SF), with high true positives. It had more difficulty distinguishing between Primary Forests (PF) and Selectively Logged Forests (SLF), with a notable number of misclassifications between these two types. This suggests the model is good at identifying SF but less precise with the other two forest types.

### Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced')
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Report:\n", classification_report(y_test, y_pred_rf, target_names=le.classes_))

# Feature Importance
importance_rf = pd.Series(rf.feature_importances_, index=X.columns)

# Plot Top 10 Features
importance_rf.nlargest(10).sort_values().plot(
    kind='barh',
    title="Top 10 Feature Importances (Decision Tree)"
)

plt.tight_layout()
plt.show()

The Random Forest model showed improved overall performance with an accuracy of 84%. It maintained high scores for SF (f1-score 0.96) and significantly improved classification for PF (f1-score 0.67) and SLF (f1-score 0.64) compared to the Decision Tree. This suggests the ensemble method is more effective in correctly identifying all three forest types.

Similar to the Decision Tree, Random Forest also highlighted `dbh` as a top feature, along with various `origin` and tree dimension related features like `ht_crown_base`, `ht_total`, and `stem_y`. The importance values are more distributed across several features compared to the single Decision Tree. This indicates that the ensemble of trees utilizes a broader set of features to make predictions.

In [ ]:
cm_rf = confusion_matrix(y_test, y_pred_rf)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=le.classes_)
disp.plot(cmap="Blues")
plt.title("Random Forest - Confusion Matrix")
plt.show()

The Random Forest confusion matrix shows improved performance across all classes compared to the Decision Tree, particularly for PF and SLF. While still having some misclassifications between PF and SLF, the numbers are lower than the Decision Tree. This demonstrates that the Random Forest's ensemble approach leads to a more robust and accurate classification across all forest types.

### k-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)

print("KNN Report:\n", classification_report(y_test, y_pred_knn, target_names=le.classes_))

In [ ]:
cm_knn = confusion_matrix(y_test, y_pred_knn)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn, display_labels=le.classes_)
disp.plot(cmap="Oranges")
plt.title("KNN - Confusion Matrix")
plt.show()

The KNN confusion matrix indicates good performance on the majority class (SF), similar to the other models. However, it shows more difficulty classifying the minority classes, particularly SLF, with a significant number of SLF instances being misclassified as SF. This suggests that while KNN is effective for the dominant class, it struggles with the less represented forest types in this dataset.

In [ ]:
# Permutation Importance
result = permutation_importance(knn, X_test_scaled, y_test, n_repeats=10, random_state=42)

# Convert to Series
importance_knn = pd.Series(result.importances_mean, index=X.columns)
importance_knn = importance_knn.sort_values(ascending=False)

# # Print or plot top 10
# print("Top 10 Important Features (kNN):\n", importance_knn.head(10))

# Optional: Plot
importance_knn.head(10).sort_values().plot(kind='barh', title="Top 10 Feature Importances (kNN - Permutation)")
plt.tight_layout()
plt.show()

The permutation importance for KNN, which measures the decrease in model score when a feature's values are randomly shuffled, shows `origin_NE` and `origin_SW` as the most important features. `ht_crown_base` and `ht_total` also appear in the top features, suggesting spatial origin and tree height are key factors for KNN's distance-based classification. The importance values are generally lower and more spread out compared to the tree-based models.

### Visualizing the Feature Importance Across the Three Models

In [ ]:
# 1. Decision Tree Importance
importance_dt = pd.Series(dt.feature_importances_, index=X.columns)
importance_dt = importance_dt.sort_values(ascending=False)

# 2. Random Forest Importance
importance_rf = pd.Series(rf.feature_importances_, index=X.columns)
importance_rf = importance_rf.sort_values(ascending=False)

# 3. Combine with kNN importance
combined_df = pd.DataFrame({
    'DecisionTree': importance_dt,
    'RandomForest': importance_rf,
    'kNN_Permutation': importance_knn
})

# Optional: Only show top 10 by average importance
combined_df['avg'] = combined_df.mean(axis=1)
top10 = combined_df.sort_values(by='avg', ascending=False).head(10)
print(top10.drop(columns='avg'))

# 4. Plot Comparison
top10.drop(columns='avg').plot(kind='barh', figsize=(10, 6), title="Top 10 Feature Importances by Model")
plt.tight_layout()
plt.show()


### **Summary and Key Findings**

This analysis explored the structural and compositional differences among primary (PF), selectively logged (SLF), and secondary (SF) forests in the Tapajos National Forest, Brazil, using a dataset of tree inventory and biometry measurements.

<br>

####**Data Preprocessing:**
The dataset was loaded and initially inspected. Missing values were identified in the `origin` column and handled by dropping the corresponding rows. A significant number of missing or unprovided values (-9999) were found in crown-related features, leading to their exclusion from model training. Duplicate checks confirmed that tree IDs are unique within each plot. Categorical features (`origin`, `genus`, `family`) were one-hot encoded, and the target variable (`type`) was label encoded. Numerical features were scaled for use with the KNN model.

<br>

####**Exploratory Data Analysis (EDA):**

* **Numerical Distributions:** Histograms revealed that many numerical features, including `dbh` and `ht_total`, exhibit right-skewed distributions, typical of forest structures with many small and fewer large trees. `density_wood` showed a more symmetrical distribution.

* **Categorical Distributions:** Bar plots highlighted that Secondary Forests (SF) represent the majority class in the dataset, indicating an imbalance. The origin analysis showed a prevalence of the "SW" origin point. The plots also identified the most abundant tree families, genera, and species, revealing differences in taxonomic composition across the forest types.

* **Tree Height and Diameter:** Analysis of `ht_total` and `dbh` showed that trees in PF and SLF generally have larger heights and diameters with higher variability compared to trees in SF, which are typically smaller and more uniform.

* **Tree Density:** Secondary Forests (SF) displayed the highest average tree density per plot and the greatest variability in density, while PF and SLF had lower, less variable densities.

* **Species Richness:** Selectively Logged Forests (SLF) and Primary Forests (PF) showed slightly higher species richness (number of unique species) compared to Secondary Forests (SF).

* **Family Composition:** The relative abundance of tree families differed across forest types. SF had a higher proportion of certain families like Salicaceae and Urticaceae, whereas PF and SLF had a higher proportion of families like Sapotaceae and Burseraceae.

* **Wood Density:** Trees in PF and SLF exhibited higher average wood densities than those in SF.

<br>

###**Models**
Three classification models (Decision Tree, Random Forest, and k-Nearest Neighbors) were trained to predict forest type.

* **Decision Tree:** Achieved 77% accuracy, performing well on SF but struggling with PF and SLF classification. `dbh` and `origin_NE` were identified as most important features.

* **Random Forest:** Showed improved performance with 84% accuracy, classifying all three forest types more effectively than the Decision Tree. `dbh` remained a key feature, along with various tree dimension and origin features.

* **k-Nearest Neighbors (KNN):** Achieved 80% accuracy, performing well on SF but showing limitations in classifying SLF, which was often misclassified as SF. Permutation importance highlighted `origin_NE` and `origin_SW` as highly important features.

<br>

The analysis reveals clear structural and compositional differences among the three forest types. Primary and selectively logged forests tend to have larger trees (height and dbh) and higher wood density compared to secondary forests. Secondary forests, while having higher overall tree density, are composed of smaller trees and a different dominant family composition. The Random Forest model proved most effective in distinguishing between these forest types based on the selected features, with dbh and geographical origin playing significant roles in classification.